# Ética en Machine Learning: Detección y Mitigación de Sesgos

## Objetivos
- Comprender los conceptos de sesgo (bias) y justicia (fairness) en ML.
- Identificar fuentes comunes de sesgo en datos y modelos.
- Utilizar herramientas para medir el sesgo en modelos de clasificación.
- Aplicar técnicas de pre-procesamiento y post-procesamiento para mitigar sesgos.
- Evaluar el trade-off entre la precisión del modelo y la justicia.

## Contexto
En esta práctica, exploraremos cómo los modelos de machine learning pueden perpetuar o incluso amplificar sesgos existentes en los datos. Trabajaremos con un dataset sintético de decisiones de scouting para evaluar si un modelo de recomendación de fichajes es justo con respecto a la nacionalidad de los jugadores.

## 1. Preparación del Entorno

### Instalación de Dependencias

In [ ]:
!pip install pandas scikit-learn plotly fairlearn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import plotly.graph_objects as go
from fairlearn.metrics import (
    MetricFrame,
    selection_rate,
    demographic_parity_difference,
    equalized_odds_difference
)
from fairlearn.postprocessing import ThresholdOptimizer
import warnings
warnings.filterwarnings('ignore')

## 2. Creación del Dataset con Sesgo Introducido

Generaremos un dataset sintético donde la nacionalidad de un jugador influye en la decisión de ficharlo, introduciendo un sesgo deliberado.

In [ ]:
np.random.seed(42)
n_players = 2000

data = {
    'age': np.random.randint(18, 30, n_players),
    'potential': np.random.uniform(60, 95, n_players),
    'performance_rating': np.random.uniform(6, 10, n_players),
    'physical_score': np.random.uniform(50, 90, n_players),
    'nationality': np.random.choice(['Local', 'Foreign'], n_players, p=[0.7, 0.3])
}

df = pd.DataFrame(data)

# Introducir sesgo: los jugadores locales tienen más probabilidad de ser fichados
score = (df['potential'] + df['performance_rating']*10 + df['physical_score']) / 3
probability = 1 / (1 + np.exp(-(score - 75)))

# Aumentar la probabilidad para jugadores locales
probability[df['nationality'] == 'Local'] *= 1.2
probability = np.clip(probability, 0, 1)

df['sign_decision'] = (probability > np.random.rand(n_players)).astype(int)

# Guardar dataset
df.to_csv('biased_scouting_decisions.csv', index=False)

print("Dataset con sesgo creado:")
print(df.head())
print("\nDistribución de decisiones por nacionalidad:")
print(df.groupby('nationality')['sign_decision'].value_counts(normalize=True))

## 3. Entrenamiento del Modelo y Evaluación Inicial

Entrenaremos un modelo de clasificación sin tener en cuenta el sesgo y evaluaremos su rendimiento.

In [ ]:
# Preparar datos
X = df.drop(['sign_decision', 'nationality'], axis=1)
y = df['sign_decision']
sensitive_features = df['nationality']

X_train, X_test, y_train, y_test, sf_train, sf_test = train_test_split(
    X, y, sensitive_features, test_size=0.3, random_state=42
)

# Entrenar modelo
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluación de precisión
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.3f}\n")
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

## 4. Medición del Sesgo con Fairlearn

Utilizaremos `fairlearn` para cuantificar el sesgo en nuestro modelo.

In [ ]:
# Métricas de justicia
metrics = {
    'accuracy': accuracy_score,
    'selection_rate': selection_rate
}

metric_frame = MetricFrame(
    metrics=metrics,
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sf_test
)

print("Métricas por grupo:")
print(metric_frame.by_group)

# Diferencia en paridad demográfica
dpd = demographic_parity_difference(y_test, y_pred, sensitive_features=sf_test)
print(f"\nDemographic Parity Difference: {dpd:.3f}")

# Diferencia en probabilidades igualadas
eod = equalized_odds_difference(y_test, y_pred, sensitive_features=sf_test)
print(f"Equalized Odds Difference: {eod:.3f}")

# Visualización
metric_frame.by_group.plot.bar(
    subplots=True,
    layout=[1, 2],
    legend=False,
    figsize=[8, 4],
    ylim=[0,1]
)

## 5. Mitigación del Sesgo con Post-procesamiento

Aplicaremos `ThresholdOptimizer` de `fairlearn` para ajustar los umbrales de decisión del modelo y mejorar la justicia.

In [ ]:
# Crear y entrenar el optimizador de umbral
postprocess_est = ThresholdOptimizer(
    estimator=model,
    constraints="demographic_parity",
    objective="accuracy_score",
    prefit=True
)

# Entrenar el optimizador
postprocess_est.fit(X_train, y_train, sensitive_features=sf_train)

# Obtener predicciones ajustadas
y_pred_post = postprocess_est.predict(X_test, sensitive_features=sf_test)

# Evaluar el modelo mitigado
print("--- Resultados después de la mitigación ---")
accuracy_post = accuracy_score(y_test, y_pred_post)
print(f"Precisión del modelo mitigado: {accuracy_post:.3f}\n")

metric_frame_post = MetricFrame(
    metrics=metrics,
    y_true=y_test,
    y_pred=y_pred_post,
    sensitive_features=sf_test
)

print("Métricas por grupo (mitigado):")
print(metric_frame_post.by_group)

dpd_post = demographic_parity_difference(y_test, y_pred_post, sensitive_features=sf_test)
print(f"\nDemographic Parity Difference (mitigado): {dpd_post:.3f}")

eod_post = equalized_odds_difference(y_test, y_pred_post, sensitive_features=sf_test)
print(f"Equalized Odds Difference (mitigado): {eod_post:.3f}")

# Visualización comparativa
metric_frame_post.by_group.plot.bar(
    subplots=True,
    layout=[1, 2],
    legend=False,
    figsize=[8, 4],
    ylim=[0,1]
)

## 6. Ejercicios Prácticos

### Ejercicio 1: Explorar Diferentes Métricas de Justicia
1. Investiga y aplica otras métricas de `fairlearn`, como `true_positive_rate_difference`.
2. Cambia el `constraint` en `ThresholdOptimizer` a `equalized_odds` y analiza los resultados. ¿Cómo cambian la precisión y la justicia?

### Ejercicio 2: Técnicas de Pre-procesamiento
1. Investiga algoritmos de pre-procesamiento para mitigar sesgos, como `Reweighing` de `fairlearn`.
2. Aplica `Reweighing` a los datos de entrenamiento y vuelve a entrenar el modelo. Compara los resultados con el post-procesamiento.

### Ejercicio 3: Análisis de Trade-off
1. Crea un gráfico que muestre la precisión del modelo vs. la métrica de justicia (e.g., `demographic_parity_difference`) para el modelo original y el mitigado.
2. Discute el "costo" de la justicia en términos de pérdida de precisión. ¿Es aceptable en este contexto?

## Reflexiones y Tareas

1. **¿Qué otras fuentes de sesgo podrían existir en datos de scouting de fútbol?** (Ej: sesgo de selección, sesgo histórico).
2. **¿Quién debería ser responsable de asegurar la justicia en los modelos de ML en un club de fútbol?**
3. **¿Cómo comunicarías los resultados de un análisis de sesgo a una audiencia no técnica?**
4. **Imagina que el modelo se usa para decidir a qué juveniles se les ofrece un contrato profesional. ¿Qué implicaciones éticas tiene esto?**

### Recursos Adicionales

- [Fairlearn Documentation](https://fairlearn.org/)
- [Google's AI Fairness Resources](https://ai.google/responsibilities/responsible-ai-practices/)
- [A Tutorial on Fairness in Machine Learning](https://arxiv.org/abs/1810.03882)

---

**¡Felicidades!** Has dado un paso importante para convertirte en un científico de datos responsable, aprendiendo a identificar, medir y mitigar sesgos en modelos de machine learning.